In [1]:
import pandas as pd
import requests
from tqdm.auto import tqdm
from bs4 import BeautifulSoup as bs
import numpy as np
import math

# Scraping the main page
Here we will scrape the main page with all the links and names of museums in Russia

In [3]:
source = requests.get('http://vrm.museum.ru/').text
soup = bs(source, 'lxml')

In [4]:
datapile = soup.find('div', class_='mlist')

In [5]:
columns = ['VRM_link', 'name', 'address', 'dates', 'budget_status', 'org_type', 'class', 'area', 'employees', 'visitors',
           'sponsors', 'storage']

In [6]:
df = pd.DataFrame(columns = columns)
df

for entry in datapile.find_all('a'):
    temp = pd.DataFrame(columns = columns, index = range(0,1)) 
    VRM_link = 'http://www.museum.ru' + entry['href']
    name = entry.text
    temp.iloc[0] = {'VRM_link': VRM_link, 'name': name}
    df = df.append(temp, ignore_index=True)
    
df.tail()

,VRM_link,name,address,dates,budget_status,org_type,class,area,employees,visitors,sponsors,storage
3024,http://www.museum.ru/M2326,Ярославский музей боевой славы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3025,http://www.museum.ru/M1592,Ярославский художественный музей,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3026,http://www.museum.ru/M3010,Ярский историко-краеведческий музей,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3027,http://www.museum.ru/M724,Ярцевский историко-краеведческий музей,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3028,http://www.museum.ru/M601,Ясногорский художественно-краеведческий музей,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Scraping each museum's page
Here we will scrape each museum's page in the database using the dataframe (df) created on the previous stage. The database isn't well-organized in html. Therefore, we will have to resort to string analysis rather than using BeautifulSoup.

First, we'lll create a list of unique entries in the database because there's no documentation for it and we will have to explore it ourselves

In the following cells we find and count entries available in database

In [6]:
entries = pd.DataFrame(columns = ['entry', 'count']) #Creating an empty df for storage and counting

In [7]:
for j in tqdm(range (0, df.shape[0])):
    
    m_source = requests.get(df.iloc[j].VRM_link).text
    m_soup = bs(m_source, 'lxml')
    m_datapile = m_soup.find('center')
    
    for i in m_datapile.find_all('b'): 
        
        if ('http' not in i.text) & (':' in i.text):
            phrase = i.text.strip().replace('*','')
            
            if (entries.entry.isin([phrase]).sum() < 1):
                temp = pd.DataFrame(columns = ['entry', 'count'], index = range(0,1)) 
                temp.iloc[0].entry = phrase
                temp['count'][0] = 1
                entries = entries.append(temp, ignore_index=True)
            else:
                try:
                    ind = entries.index[entries.entry == phrase].tolist()[0]
                    entries['count'][ind] += 1
                except:
                    n=0 

In [12]:
entries = pd.read_csv("entries_save.csv", index_col=0)

In [47]:
entries

,entry,count
0,Адрес:,3029
1,Режим работы:,3030
2,Интернет:,3029
3,Адрес дирекции:,3029
4,Организационно-правовая форма:,2904
5,Виртуальные ресурсы:,3029
6,Бюджетный статус:,2767
7,Тип организации:,2757
8,Телефон:,2709
9,Проезд:,2469


As we can see there are 99 unique entries in the base whch we define as a < b > tag on an html page. Not all of them are relevant, however they are instrumental for our future tasks

In [15]:
unique_entries = entries.entry.to_list()

We need only a few entries for our research

In [16]:
select_entries = ['Адрес:', #0
                  'Даты открытия и основания:', #1
                  'Бюджетный статус:', #2
                  'Организационно-правовая форма:', #3
                  'Тип организации:', #4
                  'Классификация организации:', #5
                  'Площади организации:', #6
                  'Количество сотрудников:', #7
                  'Среднее кол. посетителей в год:', #8
                  'Спонсоры, меценаты и грантодатели:', #9
                  'Единиц хранения:'] #10

Here is a complicated code for filling the dataframe with data

In [57]:
for i in tqdm(range (0, df.shape[0])):
    
    m_source = requests.get(df.iloc[i].VRM_link).text
    m_soup = bs(m_source, 'lxml')
    m_datapile = m_soup.find('center')
    
    s = m_datapile.text
    s = s.replace('\xa0', '')
    s = s.replace('\r', '')
    s = s.replace('\t', '')
    s = s.replace('\n', ' ')
    
    stop_points = []
    for entry in unique_entries:
        try:
            point = s.index(entry)
            stop_points.append(point)
        except:
            n=0
    stop_points.sort(key=abs)
    
    for entry in select_entries:
        try:
            start = s.index(entry) + len(entry)
            for y in stop_points:
                if start < y:
                    end = y
                    break
            
            part_of_s = ''        
            for y in range(start, end):
                part_of_s += s[y]
                
            if entry == 'Адрес:':
                df.iloc[i].address = part_of_s.strip()
            if entry == 'Даты открытия и основания:':
                df.iloc[i].dates = part_of_s.strip()
            if entry == 'Бюджетный статус:':
                df.iloc[i].budget_status = part_of_s.strip()
            if entry == 'Организационно-правовая форма:':
                df.iloc[i].org_type = part_of_s.strip()
            if entry == 'Классификация организации:':
                df.iloc[i]['class'] = part_of_s.strip()
            if entry == 'Площади организации:':
                df.iloc[i].area = part_of_s.strip()
            if entry == 'Количество сотрудников:':
                df.iloc[i].employees = part_of_s.strip()
            if entry == 'Среднее кол. посетителей в год:':
                df.iloc[i].visitors = part_of_s.strip()
            if entry == 'Спонсоры, меценаты и грантодатели:':
                df.iloc[i].sponsors = part_of_s.strip()
            if entry == 'Единиц хранения:':
                df.iloc[i].storage = part_of_s.strip()
        except:
            n=0

In [58]:
df.to_csv('df_save.csv')

In [31]:
df = pd.read_csv('df_save.csv', index_col=0)

# Refining the dataset
We will need to standartize and refine the datasat for ease of future use and uniformity of values

#### Budget and Address

In [32]:
for i in range(0, df.shape[0]):
    try:
        df.iloc[i]['address'] = df.iloc[i]['address'].strip()
        df.iloc[i]['budget_status'] = df.iloc[i]['budget_status'].strip()
    except:
        n=0
    if df.iloc[i]['budget_status'] == '':
        df.iloc[i]['budget_status'] = float('nan')

In [33]:
#df['budget_status'].unique()

In [34]:
bs_unique = ['Субъект РФ', 'Муниципальный', 'Федеральный','Общественная организация', 'Частная', 'Ведомственный', 
             'Личный']

#### Class

In [35]:
#df['class'].unique()

In [36]:
classes = pd.DataFrame(columns= ['class', 'count'])
for i in tqdm(range(0, df.shape[0])):
    if isinstance(df.iloc[i]['class'], str) == True:
        df.iloc[i]['class'] = df.iloc[i]['class'].strip()
        s = df.iloc[i]['class']
        cls = ''
        for j in range(0, len(s)):
            if (s[j] != ' ') & (s[j] != ','):
                cls += s[j]
            elif (cls != '') & (classes['class'].isin([cls]).sum() < 1):
                temp = pd.DataFrame(columns= ['class', 'count'], index = range(0,1))
                temp.iloc[0] = {'class': cls, 'count': 1}
                classes = classes.append(temp, ignore_index=True)
                cls = ''
            elif (cls != '') & (classes['class'].isin([cls]).sum() >= 1):
                ind = classes.index[classes['class'] == cls].tolist()[0]
                classes['count'][ind] += 1
                cls = ''

In [37]:
#classes.sort_values('count', ascending = False)

In [38]:
uc_names = ['историческая', 'краеведческая', 'художественная', 'персональная или мемориальная', 'естественнонаучная',
                 'науки, техники и отраслей н.х.', 'литературная', 'архитектурно-ансамблевая', 'музей-заповедник',
                 'музыкальная', 'театральная']

In [39]:
uc_vars = ['c_historical', 'c_local_history', 'c_art', 'c_pers_memorial', 'c_nat_history', 'c_sci_tech',
          'c_literary', 'c_architectural', 'c_reserve', 'c_musical', 'c_theatrical']

In [ ]:
unique_classes = pd.DataFrame(columns = ['name', 'variable'], index = range(0,len(uc_names)))
unique_classes.name = pd.Series(uc_names)
unique_classes.variable = pd.Series(uc_vars)

In [ ]:
for uc in unique_classes.variable:
    df[uc] = 0

In [ ]:
for i in tqdm(range(0, df.shape[0])):
    s = df.iloc[i]['class']
    if isinstance(s, str) == True:
        for j in range(0, unique_classes.shape[0]):
            name = unique_classes['name'][j]
            var = unique_classes['variable'][j]
            try:
                ind = s.index(name)
                end = ind + len(name)
                if s[ind:end] == name:
                    df[var][i] = 1
            except:
                n=0

C:\Users\Mvideo\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


#### Organizational type

In [ ]:
#df.org_type.unique()

In [ ]:
for i in range(0, df.shape[0]):
    if isinstance(df.iloc[i]['org_type'], str) == True:
        df.iloc[i]['org_type'] = df.iloc[i]['org_type'].strip()

C:\Users\Mvideo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
orgtype_names = ['некоммерческое учреждение', 'некоммерческая автономная', 'некоммерческое партнерство',
                 'коммерческая организация']
orgstat_vars = ['ot_ncinst', 'ot_ncaut', 'ot_ncpartner', 'ot_commercial']
orgstat = pd.DataFrame(columns = ['name', 'variable'], index = range(0,len(orgtype_names)))
orgstat.name = pd.Series(orgtype_names)
orgstat.variable = pd.Series(orgstat_vars)

In [ ]:
for ot in orgstat.variable:
    df[ot] = 0

In [ ]:
for i in tqdm(range(0, df.shape[0])):
    s = df.iloc[i]['org_type']
    if isinstance(s, str) == True:
        for j in range(0, orgstat.shape[0]):
            name = orgstat['name'][j]
            var = orgstat['variable'][j]
            try:
                ind = s.index(name)
                end = ind + len(name)
                if s[ind:end] == name:
                    df[var][i] = 1
            except:
                n=0

C:\Users\Mvideo\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


#### Area

In [ ]:
#df.area.unique()

In [ ]:
areas = pd.DataFrame(columns= ['area_type', 'count'])
for i in tqdm(range(0, df.shape[0])):
    if isinstance(df.iloc[i]['area'], str) == True:
        
        df.iloc[i]['area'] = df.iloc[i]['area'].strip()
        dump = df.iloc[i]['area']
        area_type = ''
        
        for j in range(0, len(dump)):
            if (dump[j] != ' ') & (dump[j] != ',') & (dump[j].isnumeric() == False):
                area_type += dump[j]
            elif (area_type != '') & (areas['area_type'].isin([area_type]).sum() < 1):
                temp = pd.DataFrame(columns= ['area_type', 'count'], index = range(0,1))
                temp.iloc[0] = {'area_type': area_type, 'count': 1}
                areas = areas.append(temp, ignore_index=True)
                area_type = ''
            elif (area_type != '') & (areas['area_type'].isin([area_type]).sum() >= 1):   
                ind = areas.index[areas['area_type'] == area_type].tolist()[0]
                areas['count'][ind] += 1
                area_type = ''

C:\Users\Mvideo\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [ ]:
#areas

In [ ]:
areas_unique = ['экспозиционно-выставочная', 'временных выставок', 'фондохранилищ', 'парковая']
areas_vars = ['a_exhibition', 'a_temporary', 'a_storage','a_park']
areas = pd.DataFrame(columns = ['type', 'variable'], index = range(0, 4))
areas.type = pd.Series(areas_unique)
areas.variable = pd.Series(areas_vars)
#areas

In [ ]:
#areas.iloc[0]['type']

In [ ]:
df['a_exhibition'] = float('nan')
df['a_temporary'] = float('nan')
df['a_storage'] = float('nan')
df['a_park'] = float('nan')

In [ ]:
for i in tqdm(range (0, df.shape[0])):
    s = df.iloc[i]['area']
    for j in range(0, areas.shape[0]):
        key = areas.iloc[j]['type']
        var = areas.iloc[j]['variable']
        try:
            spot = s.index(key)
            spot += len(key)
            flag = True
            result = ''
            for k in range(spot, len(s)):
                if (flag == True) & ((s[k].isnumeric() == True) or (s[k] == ',')):
                    result += s[k]
                elif s[k] == 'м':
                    flag = False
            result = result.replace(',', '.')
            df[var][i] = float(result)
        except:
            n = 0

#### Employees

In [ ]:
for i in tqdm(range(0, df.shape[0])):
    if df.iloc[i]['employees'] == '':
        df['employees'][i] = float('nan')
        
    if isinstance(df.iloc[i]['employees'], str) == True:
        df['employees'][i] = df.iloc[i]['employees'].strip()
        j = 0
        nemp = ''
        s = df.iloc[i]['employees']
        flag = False
        for j in range(0, len(s)):
            if flag == False:
                if s[j].isnumeric() == True:
                    nemp += s[j]
                elif s[j] == ',':
                    flag = True
        df['employees'][i] = int(nemp)

#### Visitors

In [ ]:
for i in tqdm(range(0, df.shape[0])):
    if isinstance(df.iloc[i]['visitors'], str) == True:
        df['visitors'][i] = df.iloc[i]['visitors'].strip()
        j = 0
        v = ''
        s = df.iloc[i]['visitors']
        flag = False
        for j in range(0, len(s)):
            if (s[j].isnumeric() == True) & (flag == False):
                v += s[j]
            elif s[j] == ',':
                flag = True
        df['visitors'][i] = int(v)

#### Storage

In [ ]:
for i in tqdm(range(0, df.shape[0])):
    if isinstance(df.iloc[i]['storage'], str) == True:
        df['storage'][i] = df.iloc[i]['storage'].strip()
        j = 0
        storage = ''
        s = df.iloc[i]['storage']
        flag = False
        for j in range(0, len(s)):
            if (s[j].isnumeric() == True) & (flag == False):
                storage += s[j]
            elif s[j] == ',':
                flag = True
        df['storage'][i] = int(storage)

#### Date of Foundation

In [ ]:
founded = 'основан:'
opened = 'открыт:'

In [ ]:
df['year_founded'] = float('nan')

In [ ]:
for i in tqdm(range(0, df.shape[0])):
    s = df.iloc[i].dates
    date = ''
    if isinstance(s, str) == True:
        try:
            ind = s.index(founded)
        except:
            ind = s.index(opened)
        for j in range(ind + 8, len(s)):
            if (s[j-4:j].isnumeric() == True) & (date == ''):
                flag = True
                date = s[j-4:j]
                df['year_founded'][i] = int(date)

#### Outliers elimination

In [ ]:
columns_in_question = ['employees', 'visitors', 'storage', 'a_exhibition', 'a_temporary', 'a_storage', 'a_park']

In [ ]:
lower_bound = 0.005
upper_bound = 0.995

In [ ]:
for col in columns_in_question:
    l = df[col].quantile([lower_bound]).loc[lower_bound]
    u = df[col].quantile([upper_bound]).loc[upper_bound]
    df[col][df[col] < l] = float('nan')
    df[col][df[col] > u] = float('nan')

# Extracting Valuable Information

First, let's save our final dataset, so we won't have to run all of the code and wait

In [217]:
df.to_excel('df_final.xlsx')

In [221]:
df[df.budget_status == 'Федеральный'].a_exhibition.mean()

3350.4544067796605

In [223]:
df[df.budget_status == 'Частная'].shape[0]

158

Let's find differences between groups of museums divided by their budget status

In [278]:
status = df.budget_status.unique().tolist()
status = status[1:len(status)]

In [288]:
findings = pd.DataFrame(columns = ['budget_status', 'sum', 'avg_a_exhibition', 'avg_a_temporary',
                                   'avg_a_storage', 'avg_a_park', 'sum_c_historical', 'sum_c_local_history', 'sum_c_art',
                                   'sum_c_pers_memorial', 'sum_c_nat_history', 'sum_c_sci_tech', 'sum_c_literary',
                                  'sum_c_architectural', 'sum_c_reserve', 'sum_c_musical', 'sum_c_theatrical',
                                   'avg_employees', 'avg_visitors', 'avg_storage'],
                       index = range(0, len(status)))

In [289]:
findings.budget_status = pd.Series(status)

In [290]:
for i in range(0, findings.shape[0]):
    findings['sum'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['budget_status'].shape[0]
    
    findings['avg_a_exhibition'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['a_exhibition'].mean()
    findings['avg_a_temporary'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['a_temporary'].mean()
    findings['avg_a_storage'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['a_storage'].mean()
    findings['avg_a_park'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['a_park'].mean()
    
    findings['sum_c_historical'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['c_historical'].sum()
    findings['sum_c_local_history'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['c_local_history'].sum()
    findings['sum_c_art'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['c_art'].sum()
    findings['sum_c_pers_memorial'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['c_pers_memorial'].sum()
    findings['sum_c_nat_history'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['c_nat_history'].sum()
    findings['sum_c_sci_tech'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['c_sci_tech'].sum()
    findings['sum_c_literary'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['c_literary'].sum()
    findings['sum_c_architectural'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['c_architectural'].sum()
    findings['sum_c_reserve'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['c_reserve'].sum()
    findings['sum_c_musical'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['c_musical'].sum()
    findings['sum_c_theatrical'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['c_theatrical'].sum()
    
    findings['avg_employees'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['employees'].mean()
    findings['avg_visitors'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['visitors'].mean()
    findings['avg_storage'][i] = df[df.budget_status == findings.iloc[i]['budget_status']]['storage'].mean()

In [291]:
findings.to_excel('findings.xlsx')

In [287]:
df['visitors'][1854] = float('nan')

C:\Users\Mvideo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Plotting the data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
sns.barplot(x='budget_status',y='employees',data=df, estimator=np.median)

# df_breakdown variables guide

| Variable      | Description                                                  |Russian translation|
|---------------|:-----                                                        |  |
|VRM_link       |A link inside VRM database to access further information      |  |
|name           |Name of the organization                                      |  |
|address        |Address of the organization                                   |  |
|date           |Date of openning of the organization                          |  |
|budget_status  |Primary source of budget for the museum | Бюджетный статус, организация, формирующая большую часть бюджета |
|c_historical   |w  | Классификация 'историческая'|
|c_local_history|w | Классификация 'краеведческая'|
|c_art          |w  | Классификация 'художественная'|
|c_pers_memorial|w  | Классификация 'персональная или мемориальная'|
|c_nat_history  |w  | Классификация 'естественнонаучная'|
|c_sci_tech     |w  | Классификация 'науки, техники и отраслей н.х.'|
|c_literary     |w  | Классификация 'литературная'|
|c_architectural|w  | Классификация 'архитектурно-ансамблевая'|
|c_reserve      |w  | Классификация 'музей-заповедник'|
|c_musical      |w  | Классификация 'музыкальная'|
|c_theatrical   |w  | Классификация 'театральная'|
|a_exibition    |w  | Площадь 'экспозиционно-выставочная'|
|a_temporary    |w  | Площадь 'временных выставок'|
|a_storage      |w  | Площадь 'фондохранилищ'|
|a_park         |w  | Площадь 'парковая'|
|ot_ncinst      |w  | Организационный тип 'некоммерческое учреждение'|
|ot_ncaut       |w  | Организационный тип 'некоммерческая автономная'|
|ot_ncpartner   |w  | Организационный тип 'некоммерческое партнерство'|
|ot_commercial  |w  | Организационный тип 'коммерческая организация'|
|visitors       |w  | Количество посетителей в год|
|employees      |w  | Количество сотрудников|
|storage        |w  | Единиц хранения|
|year_founded   |w  | Год основания|

In [ ]:
orgstat_names = ['некоммерческое учреждение', 'некоммерческая автономная', 'некоммерческое партнерство',
                 'коммерческая организация']

In [ ]:
orgstat_vars = ['ot_ncinst', 'ot_ncaut', 'ot_ncpartner', 'ot_commercial']